<a href="https://colab.research.google.com/github/Alice1304/Section4_project/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B84-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 캐글에서 데이터 가져오기

- 용량이 크기 때문에 따로 다운로드 하지 않고 캐글에서 바로 데이터를 불러오기로 함

In [61]:
# 나의 캐글 Account에서 API Token 다운 받기 = > JSON 파일 다운로드 후 캐글과 연결해주기

!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyboardInterrupt: ignored

In [ ]:
#Kaggle API를 사용하기 위해 json 파일을 ~/.kaggle로 이동

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c word2vec-nlp-tutorial

In [ ]:
!ls

In [ ]:
!unzip /content/word2vec-nlp-tutorial.zip

In [ ]:
!unzip /content/labeledTrainData.tsv.zip

In [ ]:
!unzip /content/testData.tsv.zip

In [ ]:
!unzip /content/unlabeledTrainData.tsv.zip

In [62]:
#데이터 불러오기 

In [63]:
import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
SEED = 33

In [64]:
train = pd.read_csv('/content/labeledTrainData.tsv', delimiter='\t')
test = pd.read_csv('/content/testData.tsv', delimiter='\t')

In [65]:
unlabeled_train = pd.read_csv('/content/unlabeledTrainData.tsv', delimiter='\t', error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [66]:
train.shape, test.shape, unlabeled_train.shape

((25000, 3), (25000, 2), (49998, 2))

In [67]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [68]:
 unlabeled_train.head()

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


## 2. 전처리

In [69]:
#html텍스트가 포함되어 있어 삭제
#불용어 제거

from bs4 import BeautifulSoup
import re

In [70]:
sample = train['review'][0]

In [71]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [72]:
soup = BeautifulSoup(sample, 'html.parser')

In [73]:
soup.text  #html삭제

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [74]:
clean = re.sub('[^a-zA-Z]', ' ', soup.text)  #알파벳을 제외한 문자 모두 공백처리

In [75]:
clean

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [76]:
clean.lower()

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

In [77]:
!pip install nltk
import nltk 
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [78]:
from nltk.corpus import stopwords

In [79]:
eng_stopwords = stopwords.words('english')

## 2-1 Lemmatizer

In [80]:
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
#단어를 원형으로 변환해주는 라이브러리

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [81]:
lemmatizer = WordNetLemmatizer()

In [82]:
lemmatizer.lemmatize('runs')

'run'

In [83]:
def process_lemma(sentence):
  return [lemmatizer.lemmatize(word, 'v') for word in sentence]

In [84]:
# 전처리 과정을 하나의 함수로 출력하기
# 불용어 제외하고 출력하기 & 동사의 원형으로 변환 

def preprocessing(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  clean = re.sub('[^a-zA-Z]', ' ', soup.text) 
  clean = clean.lower()
  clean = [word for word in clean.split() if word not in eng_stopwords]
  clean = process_lemma(clean)

  return ' '.join(clean)  #띄어쓰기 넣어서 조인하기 


In [85]:
all_review = pd.concat([train['review'], unlabeled_train['review'],test['review']])

In [86]:
all_review_clean = all_review.apply(preprocessing)

In [87]:
all_review_clean.head() 

0    stuff go moment mj start listen music watch od...
1    classic war worlds timothy hines entertain fil...
2    film start manager nicholas bell give welcome ...
3    must assume praise film greatest film opera ev...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

## 3. Tokenizer

In [88]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [89]:
tokenizer = Tokenizer(oov_token='<OOV>')

In [90]:
tokenizer.fit_on_texts(all_review_clean)

In [91]:
len(tokenizer.word_index)

126312

In [92]:
tokenizer.word_index['give']

32

In [93]:
for i, word in enumerate(tokenizer.word_index): #단어사전만들기 
  if i > 20:
    break
  print(word, tokenizer.word_index[word])

<OOV> 1
film 2
movie 3
one 4
make 5
like 6
see 7
get 8
time 9
good 10
character 11
go 12
watch 13
even 14
would 15
think 16
story 17
really 18
well 19
show 20
look 21


In [94]:
train_sentences = all_review_clean[:len(train)]
test_sentences = all_review_clean[-len(test):]

In [95]:
train_sentences.shape, test_sentences.shape


((25000,), (25000,))

In [96]:
train_sequences = tokenizer.texts_to_sequences(train_sentences) #문자형_숫자형 치환
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [97]:
train_sequences[0]

[397,
 12,
 463,
 11594,
 83,
 931,
 127,
 13,
 895,
 507,
 13,
 21106,
 13,
 19437,
 179,
 46,
 8,
 639,
 2250,
 66,
 16,
 18,
 469,
 3273,
 179,
 5,
 188,
 643,
 2110,
 1155,
 19437,
 58,
 4431,
 58,
 258,
 2,
 240,
 12,
 7,
 349,
 1643,
 255,
 1145,
 550,
 11594,
 59,
 773,
 2039,
 29,
 471,
 550,
 593,
 26,
 4231,
 1924,
 1032,
 175,
 420,
 1453,
 782,
 2209,
 6,
 11594,
 459,
 12,
 613,
 37,
 170,
 116,
 146,
 11594,
 34889,
 9296,
 5,
 3,
 11594,
 109,
 15,
 25,
 5,
 109,
 198,
 18,
 253,
 727,
 258,
 2,
 114,
 339,
 83,
 141,
 7788,
 3475,
 1502,
 311,
 781,
 6909,
 526,
 9123,
 785,
 593,
 1370,
 46,
 11594,
 242,
 26,
 558,
 11594,
 9785,
 505,
 12451,
 781,
 6909,
 11,
 3763,
 46,
 27,
 24,
 2666,
 593,
 413,
 8743,
 179,
 724,
 11594,
 127,
 64,
 469,
 94,
 6,
 11594,
 90,
 419,
 1905,
 130,
 1523,
 2147,
 311,
 29,
 68,
 113,
 3929,
 3388,
 36,
 2,
 22089,
 26,
 311,
 516,
 843,
 613,
 43,
 4,
 129,
 152,
 518,
 130,
 630,
 890,
 1120,
 423,
 55,
 1131,
 107,
 3,
 27,
 6,
 

In [98]:
#문장의 길이가 다르기 때문에 문장의 길이를 맞춰주는 작업 진행 

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [99]:
MAX_LENGTH = 150

In [100]:
train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, truncating='post', padding='post') #truncating = 150 단어를 넘어갈 경우 뒷부분 자르기, padding = 단어가 부족할 경우 뒤에 채워주기
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, truncating='post', padding='post') 

In [101]:
train_padded.shape,test_padded.shape

((25000, 150), (25000, 150))

## 4. train test split 

- 위에서 합쳐준 데이터들을 다시 분리하기

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
train_labels = train['sentiment']

In [104]:
x_train, x_valid, y_train, y_valid = train_test_split(train_padded, train_labels, stratify=train_labels, test_size = 0.1, random_state=SEED)

## 5. Model

- 차원을 줄여주기 위해 임베딩 진행

In [105]:
EMBEDDING_DIM= 300
VOCAB_SIZE = len(tokenizer.word_index) + 1 

In [106]:
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [108]:
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(128)),
    Dropout(0.25),
    Dense(32,activation='relu'),
    Dense(1, activation='sigmoid')
    ])

In [110]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          37893900  
                                                                 
 bidirectional (Bidirectiona  (None, 150, 256)         439296    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dense_1 (Dense)             (None, 1)                 3

In [112]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
checkpoint

In [107]:
  train_sentences = all_review_cv[:len(train)]
test_sentences = all_review_cv[-len(test):]

NameError: ignored

In [ ]:
train_labels = train['sentiment']

In [ ]:
train_labels.shape

In [ ]:
train_labels

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
SEED = 0

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, random_state = SEED)

In [ ]:
rfc.fit(train_sentences, train_labels)

In [ ]:
prediction = rfc.predict(test_sentences)

In [ ]:
prediction[:10]